In [1]:
import pandas as pd
import numpy as np
import re
import os 

In [2]:
recipes_df = pd.read_csv('./recipes.csv')
reviews_df = pd.read_csv('./reviews.csv')

In [3]:
recipes_df.head()

,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,...,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions
0,38,Low-Fat Berry Blue Frozen Dessert,1533,Dancer,PT24H,PT45M,PT24H45M,1999-08-09T21:46:00Z,Make and share this Low-Fat Berry Blue Frozen ...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,1.3,8.0,29.8,37.1,3.6,30.2,3.2,4.0,NaN,"c(""Toss 2 cups berries with sugar."", ""Let stan..."
1,39,Biryani,1567,elly9812,PT25M,PT4H,PT4H25M,1999-08-29T13:12:00Z,Make and share this Biryani recipe from Food.com.,"c(""https://img.sndimg.com/food/image/upload/w_...",...,16.6,372.8,368.4,84.4,9.0,20.4,63.4,6.0,NaN,"c(""Soak saffron in warm milk for 5 minutes and..."
2,40,Best Lemonade,1566,Stephen Little,PT5M,PT30M,PT35M,1999-09-05T19:52:00Z,This is from one of my first Good House Keepi...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,0.0,0.0,1.8,81.5,0.4,77.2,0.3,4.0,NaN,"c(""Into a 1 quart Jar with tight fitting lid, ..."
3,41,Carina's Tofu-Vegetable Kebabs,1586,Cyclopz,PT20M,PT24H,PT24H20M,1999-09-03T14:54:00Z,This dish is best prepared a day in advance to...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,3.8,0.0,1558.6,64.2,17.3,32.1,29.3,2.0,4 kebabs,"c(""Drain the tofu, carefully squeezing out exc..."
4,42,Cabbage Soup,1538,Duckie067,PT30M,PT20M,PT50M,1999-09-19T06:19:00Z,Make and share this Cabbage Soup recipe from F...,"""https://img.sndimg.com/food/image/upload/w_55...",...,0.1,0.0,959.3,25.1,4.8,17.7,4.3,4.0,NaN,"c(""Mix everything together and bring to a boil..."


In [4]:
recipes_df.iloc[0]

RecipeId                                                                     38
Name                                          Low-Fat Berry Blue Frozen Dessert
AuthorId                                                                   1533
AuthorName                                                               Dancer
CookTime                                                                  PT24H
PrepTime                                                                  PT45M
TotalTime                                                              PT24H45M
DatePublished                                              1999-08-09T21:46:00Z
Description                   Make and share this Low-Fat Berry Blue Frozen ...
Images                        c("https://img.sndimg.com/food/image/upload/w_...
RecipeCategory                                                  Frozen Desserts
Keywords                      c("Dessert", "Low Protein", "Low Cholesterol",...
RecipeIngredientQuantities              

In [5]:
reviews_df.head()

,ReviewId,RecipeId,AuthorId,AuthorName,Rating,Review,DateSubmitted,DateModified
0,2,992,2008,gayg msft,5,better than any you can get at a restaurant!,2000-01-25T21:44:00Z,2000-01-25T21:44:00Z
1,7,4384,1634,Bill Hilbrich,4,"I cut back on the mayo, and made up the differ...",2001-10-17T16:49:59Z,2001-10-17T16:49:59Z
2,9,4523,2046,Gay Gilmore ckpt,2,i think i did something wrong because i could ...,2000-02-25T09:00:00Z,2000-02-25T09:00:00Z
3,13,7435,1773,Malarkey Test,5,easily the best i have ever had. juicy flavor...,2000-03-13T21:15:00Z,2000-03-13T21:15:00Z
4,14,44,2085,Tony Small,5,An excellent dish.,2000-03-28T12:51:00Z,2000-03-28T12:51:00Z


In [6]:
reviews_df.iloc[0]

ReviewId                                                    2
RecipeId                                                  992
AuthorId                                                 2008
AuthorName                                          gayg msft
Rating                                                      5
Review           better than any you can get at a restaurant!
DateSubmitted                            2000-01-25T21:44:00Z
DateModified                             2000-01-25T21:44:00Z
Name: 0, dtype: object

In [7]:
recipe_cols = ['RecipeId', 'Name', 'CookTime', 'PrepTime', 'TotalTime', 'Description', 'Images',\
              'RecipeCategory', 'Keywords', 'RecipeIngredientQuantities', 'RecipeIngredientParts', \
              'AggregatedRating', 'Calories', 'RecipeServings', 'RecipeYield', 'RecipeInstructions']
review_cols = ['ReviewId', 'RecipeId', 'Rating', 'Review']

In [8]:
# limit recipe entries
rp_df = recipes_df[recipe_cols]
rv_df = reviews_df[review_cols]
# extract rows with reviews
rp_df = pd.DataFrame.merge(rp_df, rv_df.RecipeId, on='RecipeId').drop_duplicates('RecipeId')

In [9]:
rp_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 271674 entries, 0 to 1401962
Data columns (total 16 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   RecipeId                    271674 non-null  int64  
 1   Name                        271674 non-null  object 
 2   CookTime                    228628 non-null  object 
 3   PrepTime                    271674 non-null  object 
 4   TotalTime                   271674 non-null  object 
 5   Description                 271671 non-null  object 
 6   Images                      271673 non-null  object 
 7   RecipeCategory              271425 non-null  object 
 8   Keywords                    265248 non-null  object 
 9   RecipeIngredientQuantities  271672 non-null  object 
 10  RecipeIngredientParts       271674 non-null  object 
 11  AggregatedRating            265982 non-null  float64
 12  Calories                    271674 non-null  float64
 13  RecipeServings    

In [10]:
recipe_col_subset = ['RecipeId', 'Name', 'CookTime', 'PrepTime', 'TotalTime', 'Description', 'Images',\
              'RecipeCategory', 'Keywords', 'RecipeIngredientQuantities', 'RecipeIngredientParts', \
              'AggregatedRating', 'Calories', 'RecipeInstructions']

rp_df = rp_df.dropna(subset=recipe_col_subset)
rp_df = rp_df[rp_df.Images != 'character(0)']
rp_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 112715 entries, 0 to 1401961
Data columns (total 16 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   RecipeId                    112715 non-null  int64  
 1   Name                        112715 non-null  object 
 2   CookTime                    112715 non-null  object 
 3   PrepTime                    112715 non-null  object 
 4   TotalTime                   112715 non-null  object 
 5   Description                 112715 non-null  object 
 6   Images                      112715 non-null  object 
 7   RecipeCategory              112715 non-null  object 
 8   Keywords                    112715 non-null  object 
 9   RecipeIngredientQuantities  112715 non-null  object 
 10  RecipeIngredientParts       112715 non-null  object 
 11  AggregatedRating            112715 non-null  float64
 12  Calories                    112715 non-null  float64
 13  RecipeServings    

In [11]:
# reduce rows
rp_df = rp_df
rv_df = pd.DataFrame.merge(rv_df, rp_df.RecipeId, how='inner')

In [12]:
%%time
# This take about 35s with 10000 rows
# RecipeId as a key, list of dict as a value
rc_rv_dict = dict()
for name, df in rv_df.groupby('RecipeId'):
    # drop unnecessary column, set index and transpose (to create dict)
    df = df.drop('RecipeId', axis=1).set_index('ReviewId').T
    dict1 = df.to_dict('dict')
    rc_rv_dict[name] = dict1

CPU times: total: 8.16 s
Wall time: 57.9 s


In [13]:
# add a new column
rp_df['reviews_in_dict'] = rp_df.RecipeId.apply(lambda x: rc_rv_dict[x])

In [14]:
def map_str_to_list(string):
    #pattern = re.compile(r'\"(.+)\"')
    pattern = re.compile(r'\"([^"]+)\"')
    return pattern.findall(string)

def map_for_series(series: pd.Series):
    return series.apply(lambda i: map_str_to_list(i))

In [15]:
modify_col_list = ['Images', 'Keywords', 'RecipeIngredientQuantities', 'RecipeIngredientParts', 'RecipeInstructions']
rp_df = rp_df.apply(lambda x: map_for_series(x) if x.name in modify_col_list else x)

In [16]:
rp_df.reviews_in_dict.iat[4]

{46368: {'Rating': 5,
  'Review': 'This is a very good soup.  We served ours warm with soup crackers.  Low in calories & fat. Thanks Tracy & Jeff.'},
 118530: {'Rating': 0,
  'Review': 'hi im going to make this soup tonight doing so right now will let you know im sure i will love it thank yoiu  \r\nsassyface'},
 165407: {'Rating': 3,
  'Review': 'nice base but needed some seasoning help.  Worked great on variations of seasoning such as Italian or Soul.'},
 177583: {'Rating': 2, 'Review': 'Too bland'},
 185238: {'Rating': 0,
  'Review': 'i would like the 7 day recipe for the cabbage soup please send it to me\r\n '},
 186959: {'Rating': 4,
  'Review': 'We liked this soup. We added about three cups of water and chicken boillion cubes to stretch it farther and give it a little more flavor. '},
 804233: {'Rating': 5,
  'Review': 'VERY GOOD! in both texture and taste! awesome recipe!'},
 853747: {'Rating': 0,
  'Review': "For years, I've made a soup just like this with V8 instead of tomato j

In [17]:
rp_df.reset_index().to_csv('recipe_cleaned.csv')